In [14]:
import os
from os import listdir
# import the necessary packages
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import argparse
import cv2
import numpy as np
import PIL
from PIL import Image

In [15]:
filepath2021 = "C:\\Users\\wanzh\\Pictures\\Art\\2021 Art\\Art\\"
filepath2022 = "C:\\Users\\wanzh\\Pictures\\Art\\2022 Art\\Art\\"
list2021 = os.listdir(filepath2021)
list2022 = os.listdir(filepath2022)

In [16]:
resultfile = "ArtChromaticStorytelling2022.png"

In [17]:
list2021= list2021[:-4]
for item in list2021:
    if ".jpg" not in item:
        list2021.remove(item)
#list2021

In [18]:
list2022= list2022[:-3]
for item in list2022:
    if ".jpg" not in item:
        list2022.remove(item)

In [19]:
avgs = []

In [20]:
for image_file in list2022:
    filename = image_file.strip(".jpg")
    image_path = os.path.join(filepath2022,image_file)
    
    # load the image and convert it from BGR to RGB so that
    # we can display it with matplotlib
    image = cv2.imread(image_path)
    scale_percent = 30 # percent of original size
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)

    # resize image
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    
    
    # reshape the image to be a list of pixels
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    image = np.delete(image, np.where(image == [255, 255, 255]), axis=0)
    image = np.delete(image, np.where(image == [0, 0, 0]), axis=0)
    
    pixels = np.float32(image.reshape(-1, 3))
    
    n_colors = 1
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
    flags = cv2.KMEANS_RANDOM_CENTERS

    _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
    _, counts = np.unique(labels, return_counts=True)
    dominant = palette[np.argmax(counts)]
    #print(dominant)
    avgs.append(dominant)

    

In [21]:
avgs = np.array(avgs, dtype="int")
#print(avgs)
# grab the individual bar width and allocate memory for
# the barcode visualization
height = 250
bw = 10
barcode = np.zeros((height, len(avgs) * bw, 3),dtype="uint8")

# loop over the averages and create a single 'bar' for
# each frame average in the list
for (i, avg) in enumerate(avgs):
    #print(avg)
    cv2.rectangle(barcode, (i * bw, 0), ((i + 1) * bw,height), (int(avg[2]),int(avg[1]),int(avg[0])), -1)
# write the video barcode visualization to file and then
# display it to our screen
cv2.imwrite(resultfile, barcode)
cv2.imshow("Barcode", barcode)
cv2.waitKey(0)

-1